# Using Marker-PDF

In [ ]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered
from PIL import Image

converter = PdfConverter(
    artifact_dict=create_model_dict(),
)
rendered = converter("file_path")
text, _, images = text_from_rendered(rendered)

for key, values in images.items():
  # Save the image
  values.save(f'{key}')

with open("text.md", "w") as file:
    file.write(text)  # Write the Markdown content to the file

# Importing Libraries

In [2]:
import os 
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import re
import pickle
import hashlib
import numpy as np
from dotenv import load_dotenv
from typing import List, Tuple
from rank_bm25 import BM25Okapi
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama, OllamaEmbeddings               # ChatOllama can be imported with langchain.chat_models import ChatOllama
from langchain.chains.retrieval import create_retrieval_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema import Document, HumanMessage, AIMessage, SystemMessage

#### Load Environment Variables

In [4]:
load_dotenv()
chat_model1 = os.getenv("MODEL1")
chat_model2 = os.getenv("MODEL2")
embed_model = os.getenv("EMBED_MODEL")
max_token_prompt_len = os.getenv("MODEL1_MAX_TOKEN_PROMPT_LEN")
max_token_predict_len = os.getenv("MODEL1_MAX_TOKEN_PREDICT_LEN")

# Contextual Retrieval

In [4]:
class ContextualRetrieval:
    def __init__(self):

        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
        self.embeddings = OllamaEmbeddings(model=embed_model)
        self.llm1 = ChatOllama(base_url='http://localhost:11434', model=chat_model1, num_ctx=max_token_prompt_len, num_predict=max_token_predict_len)
        self.llm2 = ChatOllama(base_url='http://localhost:11434', model=chat_model2, temperature=0.7, num_predict=1024)

    def process_document(self, document: str) -> Tuple[List[Document], List[Document]]:

        chunks = self.text_splitter.create_documents([document])
        contextualized_chunks = self._create_contextualized_chunks(document, chunks)
        return chunks, contextualized_chunks

    def _create_contextualized_chunks(self, document: str, chunks: List[Document]) -> List[Document]:

        contextualized_chunks = []
        for chunk in chunks:
            context = self._generate_context(document, chunk.page_content)    # chunk have .page_content and .metadata attrs. so use chunk.page_content not chunk for only text
            contextualized_content = f"{context}\n\n{chunk.page_content}"
            contextualized_chunks.append(Document(page_content=contextualized_content, metadata=chunk.metadata))
        return contextualized_chunks

    def _generate_context(self, document: str, chunk: str) -> str:
        
        prompt = ChatPromptTemplate.from_template("""
        You are an AI assistant specializing in understanding Research or Scientific Papers. Your task is to provide brief, relevant context for a chunk of text from the given paper.
        Here is the Scientific Paper:
        <document>
        {document}
        </document>

        Here is the chunk:
        <chunk>
        {chunk}
        </chunk>

        Consider the following guidelines:
        1. Identify the main research topic or concept discussed in the chunk (e.g., methodology, results, hypothesis, experiment).
        2. Highlight any key findings, data, or statistics that provide important context to the research (e.g., results of experiments, significant correlations, p-values, etc.).
        3. Mention any relevant time periods or specific conditions (e.g., time frames for experiments, population sample, study duration).
        4. Do not use phrases like "This chunk discusses" or "This section provides". Instead, directly state the relevant context, focusing on the research's significance and findings.

        Answer only with the succinct context in 2-3 sentences and nothing else following the guidelines.
                                                  
        Context:
        """)
        messages = prompt.format_messages(document=document, chunk=chunk)
        response = self.llm1.invoke(messages)
        return response.content

    def create_vectorstores(self, chunks: List[Document]) -> FAISS:

        faiss_db = FAISS.from_documents(documents=chunks, embedding=self.embeddings)
        index_path = "faiss_index.index"
        faiss_db.save_local(index_path)                                                 # Save the FAISS index to the current dir with name faiss_index.index
        print(f"Vectordb saved successfully")

        return faiss_db

    def create_bm25_index(self, chunks: List[Document]) -> BM25Okapi:

        tokenized_chunks = [chunk.page_content.split() for chunk in chunks]
        bm25_index = BM25Okapi(tokenized_chunks)
        save_path = "bm25_index.pkl"                                                    # save_path must be file name with .pkl extension not directory
                                                                                        # Save the BM25 index to a local file using pickle
        with open(save_path, 'wb') as f:
            pickle.dump(bm25_index, f)
        
        print(f"BM25 index saved locally at {save_path}")
        
        return bm25_index

    @staticmethod
    def generate_cache_key(document: str) -> str:
        """ Generate a cache key for a document for faster loading. """
        return hashlib.md5(document.encode()).hexdigest()

    def generate_answer(self, query: str, relevant_chunks: List[str]) -> str:

        prompt = ChatPromptTemplate.from_template("""
        Based on the following information, please provide a concise and accurate answer to the question.
        If the information is not sufficient to answer the question, say so.

        Question: {query}

        Relevant information:
        {chunks}

        Answer:
        """)
        messages = prompt.format_messages(query=query, chunks="\n\n".join(relevant_chunks))
        response = self.llm2.invoke(messages)
        return response.content

#### Loading Required Document

In [5]:
with open('Test.md', 'r') as file:
    document = file.read()

pattern = r'!\[\]\([^)]*\)'                 # Regex pattern to remove markdown image syntax ![](_image_name)
document = re.sub(pattern, '', document)

In [16]:
cr = ContextualRetrieval()
chunks, contextualized_chunks = cr.process_document(document)

Character recognition using deep learning algorithms.

OCR technology generally tends to recognition of characters, words and to compute them such that determine the information and to change into computerized character.As OCR combines with most of the research topics such signal processing, Computer graphics etc. Optical Character recognition is mechanism of conversion of images of different characters into the machine encoded text or text-imposed image. As OCR is widely used for different applications such as automatic data entry for different uses as for example passports, computerized receipts for different purposes, business cards etc. Asearly versions are needed to trained on different images worked on the single font later different fonts of images are used. The variety of image file format inputs are capable of high-level fonts


#### View the Contextualized chunks

In [20]:
print(contextualized_chunks[10])

page_content='The main concept discussed is Artificial Intelligence (AI) methodologies, specifically computational intelligence. This includes neural networks, fuzzy systems, and evolutionary computations as notable methods used in AI. The focus is on their application to deep learning algorithms, particularly CNNs for character recognition.

a varietyof methodologiescalledcomputationalintelligence,including neural networks, fuzzy systems, evolutionary computations, and other computational models. Although the methodsusedinAI aredifferent, the two main methodologies are top-down and bottom-up. While thetop-down approach believes in creating computer programs accordingto the operation of the human brain, and outlines of the system are established andspecified, the bottom-up approach is realized by providingsimilar electronicreplication to the neural networks of the human brain.'


#### Create and store vecotordb and bm25 index

In [ ]:
contextualized_vectorstore = cr.create_vectorstores(contextualized_chunks)
contextualized_retriever = contextualized_vectorstore.as_retriever(search_kwargs={"k": 4})      # Has search_type arg. which is similarity by def, can be mmr, etc.

contextualized_bm25_index = cr.create_bm25_index(contextualized_chunks)

cache_key = cr.generate_cache_key(document)

#### Loading Vectordb and BM25 index

In [62]:
cr = ContextualRetrieval()
contextualized_vectorstore = FAISS.load_local("faiss_index.index", embeddings=OllamaEmbeddings(model=embed_model), allow_dangerous_deserialization=True)
contextualized_retriever = contextualized_vectorstore.as_retriever(search_kwargs={"k": 4})

with open("bm25_index.pkl", 'rb') as f:
    contextualized_bm25_index = pickle.load(f)

#### Question Answering

In [22]:
# Example queries related to financial information
queries = [
        "What is AI? Who defined the term AI and in which year?",
        "What is deep learning in context of AI?",
        "Who have implemented this Research Paper?",
        "Give a brief summary of Introduction section.",
        "What is the main objective of this research?",
        ]

for query in queries:
        print(f"\nQuery: {query}")

        # contextualized_vector_results = contextualized_vectorstore.similarity_search(query, k=3)                                     # Retrieve from contextualized vectorstore
        contextualized_vector_results = contextualized_retriever.invoke(input=query)

        contextualized_tokenized_query = query.split()
        contextualized_bm25_results = contextualized_bm25_index.get_top_n(contextualized_tokenized_query, contextualized_chunks, n=3)  # Retrieve from contextualized BM25

        # Generate answers
        contextualized_vector_answer = cr.generate_answer(query, [doc.page_content for doc in contextualized_vector_results])
        contextualized_bm25_answer = cr.generate_answer(query, [doc.page_content for doc in contextualized_bm25_results])

        print("\nContextualized Vector Search Results:")
        for i, doc in enumerate(contextualized_vector_results, 1):
            print(f"{i}. {doc.page_content}")

        print("\nContextualized Vector Search Answer:")
        print(contextualized_vector_answer)

        print("\n" + "-"*100)

        print("\nContextualized BM25 Search Results:")
        for i, doc in enumerate(contextualized_bm25_results, 1):
            print(f"{i}. {doc.page_content}")

        print("\nContextualized BM25 Search Answer:")
        print(contextualized_bm25_answer)

        print("\n" + "="*50)


Query: What is AI? Who defined the term AI and in which year?

Contextualized Vector Search Results:
1. The main concept discussed is Artificial Intelligence (AI) methodologies, specifically computational intelligence. This includes neural networks, fuzzy systems, and evolutionary computations as notable methods used in AI. The focus is on their application to deep learning algorithms, particularly CNNs for character recognition.

a varietyof methodologiescalledcomputationalintelligence,including neural networks, fuzzy systems, evolutionary computations, and other computational models. Although the methodsusedinAI aredifferent, the two main methodologies are top-down and bottom-up. While thetop-down approach believes in creating computer programs accordingto the operation of the human brain, and outlines of the system are established andspecified, the bottom-up approach is realized by providingsimilar electronicreplication to the neural networks of the human brain.
2. The Research top

#### Code for Creating a Retrieval Chain

In [41]:
prompt = ChatPromptTemplate.from_template("""
Based on the following information, please provide a concise and accurate answer to the question.
If the information is not sufficient to answer the question, say so.

Question: 
{input}

Relevant information:
{context}

Don't start the answer with "Based on the provided information or context". Just provide the answer directly.                                          
Answer:
""")
llm = ChatOllama(base_url='http://localhost:11434', model=chat_model2, num_predict=1024)

document_chain=create_stuff_documents_chain(llm,prompt)
retrieval_chain=create_retrieval_chain(contextualized_retriever,document_chain)
response = retrieval_chain.invoke({"input": "Explain what is AI? (5 marks)"})
print(response['answer'])

Artificial Intelligence (AI) is a simulation of human intelligence processed by computers, referring to the science and engineering of creating intelligent machines. It encompasses various technologies and algorithms, including machine learning, neural networks, expert systems, and deep learning. AI aims to mimic human behavior, such as cognition, problem-solving, and learning, and has numerous applications in fields like medical, space, robotics, and military.


For this chain, we have to pass our query with the name 'input' and chunks to be replaced by 'context'.

# Conversational QnA

In [204]:
class ConverasationQnA:
    def __init__(self):
        self.llm = ChatOllama(base_url='http://localhost:11434', model=chat_model2, num_predict=1024)
        self.embeddings = OllamaEmbeddings(model=embed_model)
        self.prompt = ChatPromptTemplate.from_template("""
            Your name is Bob and you are a helpful AI assistant. Don't restrict yourself to context given below you can use your own knowledge too.
            Also don't tell who you are to the user.
            Context: {context}
            Question: {question} 
            Answer:
            """)
        document = Document(
            page_content=SystemMessage("You are Bob a helpful AI assistant.").content, 
            metadata={"role": "system"}
            )
        self.faiss_vectorstore = FAISS.from_documents([document], self.embeddings)
        print("FAISS vector store created.")

    def give_response(self, user_input : str) -> str:
        message = [HumanMessage(user_input)]
        result = self._get_vector_results(message)
        prompt = self._get_prompt_template(context=result, question=user_input)
        response = self.llm.invoke(prompt)
        document = self._generate_document(message, response)
        self.faiss_vectorstore.add_documents([document])
        return response.content
    
    def _get_prompt_template(self, question, context=None):
        return self.prompt.format_messages(context=context, question=question)
    
    def _generate_document(self, message, response) -> Document:
        document = Document(
            page_content=f"Query : {message[0].content}\n\nAnswer : {response.content}", 
            metadata={"role": "human"}
            )
        return document
    
    def _get_vector_results(self, message):
        vector_results = self.faiss_vectorstore.similarity_search_with_score(message[0].content, k=3, kwargs={"score_threshold": 0.65})
        result = ""
        for v,score in vector_results: 
            if (score > 0.65):
                result += v.page_content + "\n"
        if (result == ""):
            return None
        return result
    
    def get_vectordb(self) -> FAISS:
        return self.faiss_vectorstore

# We use .invoke() method only to pass the query to LLM don't just directly pass it.
cqa = ConverasationQnA()

FAISS vector store created.


In [219]:
db = cqa.get_vectordb()
db.save_local("conversation")
db.index.ntotal

6

In [217]:
response = cqa.give_response("Where do i study?")

In [218]:
print(response)

Nice try, Rohan! Your location isn't something I can provide an answer for. However, I can tell you that as a student at MIT, you have access to some of the best educational resources and facilities in the world. From state-of-the-art labs to cutting-edge research centers, you're surrounded by opportunities to learn and grow.

If you need help finding information about specific buildings or locations on campus, I'd be happy to try and assist you with that!


# Summarization

In [1]:
document = """Fair Join Pattern Matching for Actors
Document
https://doi.org/10.4230/LIPIcs.ECOOP.2024.17
Export XML
Export ACM-XML
Export DOAJ-XML
Export Schema.org
Export BibTeX
Fair Join Pattern Matching for Actors
Authors
Philipp Haller ,
Ayman Hussein ,
Hernán Melgratti ,
Alceste Scalas ,
Emilio Tuosto
Part of:
Volume:
38th European Conference on Object-Oriented Programming (ECOOP 2024)
Part of:
Series:
Leibniz International Proceedings in Informatics (LIPIcs)
Part of:
Conference:
European Conference on Object-Oriented Programming (ECOOP)
License:
Creative Commons Attribution 4.0 International license
Publication Date: 2024-09-12
PDF
File
LIPIcs.ECOOP.2024.17.pdf
Filesize: 1.25 MB
28 pages
Document Identifiers
DOI: 10.4230/LIPIcs.ECOOP.2024.17
URN: urn:nbn:de:0030-drops-208663
Author Details
Philipp Haller
KTH Royal Institute of Technology, Stockholm, Sweden
Ayman Hussein
Technical University of Denmark, Lyngby, Denmark
Hernán Melgratti
University of Buenos Aires & Conicet, Argentina
Alceste Scalas
Technical University of Denmark, Lyngby, Denmark
Emilio Tuosto
Gran Sasso Science Institute, L'Aquila, Italy
Funding
Hussein, Ayman: Research supported by the Horizon Europe grant 101093006 (TaRDIS).
Scalas, Alceste: Research partly supported by the Horizon Europe grant 101093006 (TaRDIS).
Tuosto, Emilio: Research partly supported by the EU H2020 RISE programme under the Marie Skłodowska-Curie grant agreement No 778233, the PRIN PNRR project DeLICE (P20223T2MF), by the MUR dipartimento di eccellenza, and by PNRR MUR project VITALITY (ECS00000041), Spoke 2 ASTRA - Advanced Space Technologies and Research Alliance.
Acknowledgements
This work was inspired by the group discussion on "Join patterns / synchronisation - the next generation" [Dezani et al., 2022] at the Dagstuhl Seminar 21372; we thank the organisers of the meeting and Schloss Dagstuhl - Leibniz Center for Informatics for making this work possible. We thank Omar Inverso for the technical support he provided for our experimental evaluation, Roland Kuhn for fruitful discussions on the shop floor use case, António Ravara for some useful suggestions, and Antoine Sébert for an implementation of join patterns using Scala 3 macros [Sébert, 2022]. We thank the anonymous reviewers for their comments and suggestions.
Cite As Get BibTex
Philipp Haller, Ayman Hussein, Hernán Melgratti, Alceste Scalas, and Emilio Tuosto. Fair Join Pattern Matching for Actors. In 38th European Conference on Object-Oriented Programming (ECOOP 2024). Leibniz International Proceedings in Informatics (LIPIcs), Volume 313, pp. 17:1-17:28, Schloss Dagstuhl – Leibniz-Zentrum für Informatik (2024)
https://doi.org/10.4230/LIPIcs.ECOOP.2024.17
BibTex
@InProceedings{haller_et_al:LIPIcs.ECOOP.2024.17,
author =	{Haller, Philipp and Hussein, Ayman and Melgratti, Hern\'{a}n and Scalas, Alceste and Tuosto, Emilio},
title =	{{Fair Join Pattern Matching for Actors}},
booktitle =	{38th European Conference on Object-Oriented Programming (ECOOP 2024)},
pages =	{17:1--17:28},
series =	{Leibniz International Proceedings in Informatics (LIPIcs)},
ISBN =	{978-3-95977-341-6},
ISSN =	{1868-8969},
year =	{2024},
volume =	{313},
editor =	{Aldrich, Jonathan and Salvaneschi, Guido},
publisher =	{Schloss Dagstuhl -- Leibniz-Zentrum f{\"u}r Informatik},
address =	{Dagstuhl, Germany},
URL =		{https://drops.dagstuhl.de/entities/document/10.4230/LIPIcs.ECOOP.2024.17},
URN =		{urn:nbn:de:0030-drops-208663},
doi =		{10.4230/LIPIcs.ECOOP.2024.17},
annote =	{Keywords: Concurrency, join patterns, join calculus, actor model}
}
@InProceedings{haller_et_al:LIPIcs.ECOOP.2024.17,
author =	{Haller, Philipp and Hussein, Ayman and Melgratti, Hern\'{a}n and Scalas, Alceste and Tuosto, Emilio},
title =	{{Fair Join Pattern Matching for Actors}},
booktitle =	{38th European Conference on Object-Oriented Programming (ECOOP 2024)},
pages =	{17:1--17:28},
series =	{Leibniz International Proceedings in Informatics (LIPIcs)},
ISBN =	{978-3-95977-341-6},
ISSN =	{1868-8969},
year =	{2024},
volume =	{313},
editor =	{Aldrich, Jonathan and Salvaneschi, Guido},
publisher =	{Schloss Dagstuhl -- Leibniz-Zentrum f{\"u}r Informatik},
address =	{Dagstuhl, Germany},
URL =		{https://drops.dagstuhl.de/entities/document/10.4230/LIPIcs.ECOOP.2024.17},
URN =		{urn:nbn:de:0030-drops-208663},
doi =		{10.4230/LIPIcs.ECOOP.2024.17},
annote =	{Keywords: Concurrency, join patterns, join calculus, actor model}
}
Close
Copy BibTex To Clipboard
Abstract
Join patterns provide a promising approach to the development of concurrent and distributed message-passing applications. Several variations and implementations have been presented in the literature - but various aspects remain under-explored: in particular, how to specify a suitable notion of message matching, how to implement it correctly and efficiently, and how to systematically evaluate the implementation performance.
In this work we focus on actor-based programming, and study the application of join patterns with conditional guards (i.e., the most expressive and challenging version of join patterns in literature). We formalise a novel specification of fair and deterministic join pattern matching, ensuring that older messages are always consumed if they can be matched. We present a stateful, tree-based join pattern matching algorithm and prove that it correctly implements our fair and deterministic matching specification. We present a novel Scala 3 actor library (called JoinActors) that implements our join pattern formalisation, leveraging macros to provide an intuitive API. Finally, we evaluate the performance of our implementation, by introducing a systematic benchmarking approach that takes into account the nuances of join pattern matching (in particular, its sensitivity to input traffic and complexity of patterns and guards).
Subject Classification
ACM Subject Classification
Software and its engineering → Formal language definitions
Software and its engineering → Domain specific languages
Software and its engineering → Concurrent programming languages
Software and its engineering → Distributed programming languages
Theory of computation → Process calculi
Keywords
Concurrency
join patterns
join calculus
actor model
Metrics
Access Statistics
Total Accesses (updated on a weekly basis)
0
PDF Downloads
0
Metadata Views
Supplementary Materials
Software (ECOOP 2024 Artifact Evaluation approved artifact)
https://doi.org/10.4230/DARTS.10.2.8
References
Gul A. Agha. ACTORS - a model of concurrent computation in distributed systems. In MIT Press series in artificial intelligence, 1986.
Nick Benton, Luca Cardelli, and Cédric Fournet. Modern concurrency abstractions for C#. ACM Trans. Program. Lang. Syst., 26(5):769-804, 2004. URL: https://doi.org/10.1145/1018203.1018205.
Oliver Bračevac, Nada Amin, Guido Salvaneschi, Sebastian Erdweg, Patrick Eugster, and Mira Mezini. Versatile event correlation with algebraic effects. Proc. ACM Program. Lang., 2(ICFP), July 2018. URL: https://doi.org/10.1145/3236762.
Mariangiola Dezani, Roland Kuhn, Sam Lindley, and Alceste Scalas. Behavioural Types: Bridging Theory and Practice (Dagstuhl Seminar 21372). Dagstuhl Reports, 11(8):52-75, 2022. URL: https://doi.org/10.4230/DagRep.11.8.52.
Fabrice Le Fessant and Luc Maranget. Compiling join-patterns. Electronic Notes in Theoretical Computer Science, 16(3):205-224, 1998. URL: https://doi.org/10.1016/S1571-0661(04)00143-4.
Charles Forgy. Rete: a fast algorithm for the many pattern/many object pattern match problem. Expert Systems, pages 324-341, 1991.
Cédric Fournet, Fabrice Le Fessant, Luc Maranget, and Alan Schmitt. Jocaml: A language for concurrent distributed and mobile programming. In Johan Jeuring and Simon L. Peyton Jones, editors, Advanced Functional Programming, 4th International School, AFP 2002, Oxford, UK, August 19-24, 2002, Revised Lectures, volume 2638 of Lecture Notes in Computer Science, pages 129-158. Springer, 2002. URL: https://doi.org/10.1007/978-3-540-44833-4_5.
Cédric Fournet and Georges Gonthier. The reflexive CHAM and the join-calculus. In Hans-Juergen Boehm and Guy L. Steele Jr., editors, Conference Record of POPL'96: The 23rd ACM SIGPLAN-SIGACT Symposium on Principles of Programming Languages, Papers Presented at the Symposium, St. Petersburg Beach, Florida, USA, January 21-24, 1996, pages 372-385. ACM Press, 1996. URL: https://doi.org/10.1145/237721.237805.
Cédric Fournet, Georges Gonthier, Jean-Jacques Lévy, Luc Maranget, and Didier Rémy. A calculus of mobile agents. In CONCUR'96: Concurrency Theory: 7th International Conference Pisa, Italy, August 26-29, 1996 Proceedings 7, pages 406-421. Springer, 1996.
Rosita Gerbo and Luca Padovani. Concurrent Typestate-Oriented Programming in Java. In Francisco Martins and Dominic Orchard, editors, Proceedings Programming Language Approaches to Concurrency- and Communication-cEntric Software, PLACES@ETAPS 2019, Prague, Czech Republic, 7th April 2019, volume 291 of EPTCS, pages 24-34, 2019. URL: https://doi.org/10.4204/EPTCS.291.3.
Rob Van Glabbeek and Peter Höfner. Progress, justness, and fairness. ACM Comput. Surv., 52(4), August 2019. URL: https://doi.org/10.1145/3329125.
Philipp Haller and Tom Van Cutsem. Implementing joins using extensible pattern matching. In Doug Lea and Gianluigi Zavattaro, editors, Coordination Models and Languages, 10th International Conference, COORDINATION 2008, Oslo, Norway, June 4-6, 2008. Proceedings, volume 5052 of Lecture Notes in Computer Science, pages 135-152. Springer, 2008. URL: https://doi.org/10.1007/978-3-540-68265-3_9.
Shams M. Imam and Vivek Sarkar. Savina - an actor benchmark suite: Enabling empirical evaluation of actor libraries. In Proceedings of the 4th International Workshop on Programming Based on Actors Agents & Decentralized Control, AGERE! '14, pages 67-80, New York, NY, USA, 2014. Association for Computing Machinery. URL: https://doi.org/10.1145/2687357.2687368.
G. Stewart Von Itzstein and Mark Jasiunas. On implementing high level concurrency in Java. In Amos Omondi and Stanislav Sedukhin, editors, Advances in Computer Systems Architecture, 8th Asia-Pacific Conference, ACSAC 2003, Aizu-Wakamatsu, Japan, September 23-26, 2003, Proceedings, volume 2823 of Lecture Notes in Computer Science, pages 151-165. Springer, 2003. URL: https://doi.org/10.1007/978-3-540-39864-6_13.
Eryk Lagun. Evaluation and implementation of match algorithms for rule-based multi-agent systems using the example of jadex. MSc Thesis, University of Hamburg, 2009. URL: https://swa.informatik.uni-hamburg.de/files/abschlussarbeiten/Diplomarbeit_Eryk_Lagun.pdf.
Qin Ma and Luc Maranget. Compiling pattern matching in join-patterns. In International Conference on Concurrency Theory, pages 417-431. Springer, 2004. URL: https://doi.org/10.1007/978-3-540-28644-8_27.
Luc Maranget. Compiling pattern matching to good decision trees. In Eijiro Sumii, editor, Proceedings of the ACM Workshop on ML, 2008, Victoria, BC, Canada, September 21, 2008, pages 35-46. ACM, 2008. URL: https://doi.org/10.1145/1411304.1411311.
Daniel P. Miranker. TREAT: a new and efficient match algorithm for AI production systems. PhD thesis, Columbia University, USA, 1987. UMI Order No. GAX87-10209.
Martin Odersky. Functional nets. In European Symposium on Programming, pages 1-25. Springer, 2000. URL: https://doi.org/10.1007/3-540-46425-5_1.
Hubert Plociniczak and Susan Eisenbach. Jerlang: Erlang with joins. In Coordination Models and Languages: 12th International Conference, COORDINATION 2010, Amsterdam, The Netherlands, June 7-9, 2010. Proceedings 12, pages 61-75. Springer, 2010.
Humberto Rodriguez Avila. Orchestration of Actor-Based Languages for Cyber-Physical Systems. PhD thesis, Vrije Universiteit Brussel, 2021.
Humberto Rodríguez-Avila, Joeri De Koster, and Wolfgang De Meuter. Advanced join patterns for the actor model based on CEP techniques. Art Sci. Eng. Program., 5(2):10, 2021. URL: https://doi.org/10.22152/programming-journal.org/2021/5/10.
Claudio V. Russo. The Joins concurrency library. In Michael Hanus, editor, Practical Aspects of Declarative Languages, 9th International Symposium, PADL 2007, Nice, France, January 14-15, 2007, volume 4354 of Lecture Notes in Computer Science, pages 260-274. Springer, 2007. URL: https://doi.org/10.1007/978-3-540-69611-7_17.
Claudio V. Russo. Join patterns for visual basic. In Gail E. Harris, editor, Proceedings of the 23rd Annual ACM SIGPLAN Conference on Object-Oriented Programming, Systems, Languages, and Applications, OOPSLA 2008, October 19-23, 2008, Nashville, TN, USA, pages 53-72. ACM, 2008. URL: https://doi.org/10.1145/1449764.1449770.
Antoine Louis Thibaut Sébert. Join-patterns for the actor model in Scala 3 using macros. Master’s thesis, DTU Department of Applied Mathematics and Computer Science, 2022. Available at URL: https://findit.dtu.dk/en/catalog/62f83d3680aa6403a4ccc0ab.
Martin Sulzmann, Edmund S. L. Lam, and Peter Van Weert. Actors with multi-headed message receive patterns. In Doug Lea and Gianluigi Zavattaro, editors, Coordination Models and Languages, 10th International Conference, COORDINATION 2008, Oslo, Norway, June 4-6, 2008. Proceedings, volume 5052 of Lecture Notes in Computer Science, pages 315-330. Springer, 2008. URL: https://doi.org/10.1007/978-3-540-68265-3_20.
Aaron Joseph Turon and Claudio V. Russo. Scalable join patterns. In Cristina Videira Lopes and Kathleen Fisher, editors, Proceedings of the 26th Annual ACM SIGPLAN Conference on Object-Oriented Programming, Systems, Languages, and Applications, OOPSLA 2011, part of SPLASH 2011, Portland, OR, USA, October 22 - 27, 2011, pages 575-594. ACM, 2011. URL: https://doi.org/10.1145/2048066.2048111.
Louise Van Verre, Humberto Rodríguez-Avila, Jens Nicolay, and Wolfgang De Meuter. Florence: A hybrid logic-functional reactive programming language. Proceedings of the 9th ACM SIGPLAN International Workshop on Reactive and Event-Based Languages and Systems, 2022.
Questions / Remarks / Feedback
X
Feedback for Dagstuhl Publishing
Send
Thanks for your feedback!
Feedback submitted
OK
Could not send message
Please try again later or send an E-mail
OK
© 2023-2024 Schloss Dagstuhl – LZI GmbH
About DROPS
Imprint
Privacy
Contact
"""

from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
llm = ChatOllama(base_url='http://localhost:11434', model="llama3.2:3b", num_predict=1024, num_ctx=8192)
prompt = ChatPromptTemplate.from_template("""
Based on the following information, please provide a concise summary of abstract in the following text.
    Docuemnt: {context}
    """)
response = llm.invoke(prompt.format_messages(context=document))

In [3]:
print(response.content)

Here is a concise summary of the abstract:

The authors propose a novel approach to join pattern matching for actor-based programming, ensuring fair and deterministic matching of messages. They formalize a specification for fair and deterministic join pattern matching, develop a stateful tree-based algorithm, and implement it in a Scala 3 actor library called JoinActors. The authors also introduce a systematic benchmarking approach to evaluate the performance of their implementation, taking into account the nuances of join pattern matching.
